You just got hired as the first and only data practitioner at a small business experiencing exponential growth. The company needs more structured processes, guidelines, and standards. Your first mission is to structure the human resources data. The data is currently scattered across teams and files and comes in various formats: Excel files, CSVs, JSON files...

You'll work with the following data in the `datasets` folder:
- __Office addresses__ are currently saved in `office_addresses.csv`. If the value for office is `NaN`, then the employee is remote. 
- __Employee addresses__ are saved on the first tab of `employee_information.xlsx`.
- __Employee emergency contacts__ are saved on the second tab of `employee_information.xlsx`; this tab is called `emergency_contacts`. However, this sheet was edited at some point, and the headers were removed! The HR manager let you know that they should be: `employee_id`, `last_name`, `first_name`, `emergency_contact`, `emergency_contact_number`, and `relationship`.
- __Employee roles, teams, and salaries__ have been exported from the company's human resources management system into a JSON file titled `employee_roles.json`. Here are the first few lines of that file (the first key is Employee ID):
```
{"A2R5H9":
  {
    "title": "CEO",
    "monthly_salary": "$4500",
    "team": "Leadership"
  },
 ...
}
```

In [115]:
import pandas as pd

# Set display options to show all columns and rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 100)

# Define file paths as variables for easier readability
offices_file = 'datasets/office_addresses.csv'
employee_info_file = 'datasets/employee_information.xlsx'
employee_roles_file = 'datasets/employee_roles.json'

# Import data into DataFrames with clear variable names
offices = pd.read_csv(offices_file)

employee_address = pd.read_excel(employee_info_file, sheet_name=0, header=0,
                                 names=["employee_id", "last_name", "first_name","employee_country",                                                                    "employee_city","employee_street","employee_street_number"])


employee_emergency = pd.read_excel(employee_info_file, sheet_name=1, header=None,
                                   names=['employee_id', 'last_name', 'first_name', 'emergency_contact',
                                          'emergency_contact_number', 'relationship'])

employee_data = pd.read_json(employee_roles_file, orient='index').reset_index().rename(columns={'index': 'employee_id'})

#Print DataFrames with clear labels and formatting
print(f"Offices:\n{offices.head()}\n")
print(f"Employee Address:\n{employee_address.head()}\n")
print(f"Employee Emergency:\n{employee_emergency.head()}\n")
print(f"Employee Data:\n{employee_data.head()}\n")



Offices:
          office office_country    office_city   office_street  office_street_number
0  Leuven Office             BE         Leuven  Martelarenlaan                    38
1     ESB Office             US  New York City    Fifth Avenue                   350
2  WeWork Office             GB         London      Old Street                   207

Employee Address:
  employee_id last_name first_name employee_country employee_city      employee_street  \
0      A2R5H9    Hunman        Jax               BE        Leuven          Grote Markt   
1      H8K0L6      Siff       Tara               GB        London         Baker Street   
2      G4R7V0     Sagal      Gemma               US      New-York         Perry Street   
3      M1Z7U9    Coates        Tig               FR         Paris  Rue de l'Université   

   employee_street_number  
0                       9  
1                     221  
2                      66  
3                       7  

Employee Emergency:
  employee_id last_n

In [116]:
df_employees = employee_address.merge(employee_emergency, how="left", on="employee_id")

# Merge df_employees with df_employee_roles
df_employees = df_employees.merge(employee_data, how="left", left_on="employee_id", 
                                  right_on="employee_id")


# Merge df_employees with df_office_adresses
df_employees = df_employees.merge(offices, how="left",
                                  left_on="employee_country", right_on="office_country")


# Drop the duplicate columns
df_employees_renamed = df_employees.drop(["last_name_y", "first_name_y"], axis=1)


df_employees_renamed = df_employees_renamed.rename(columns={'last_name_x': 'last_name', 'first_name_x': 'first_name'})

df_employees_renamed

,employee_id,last_name,first_name,employee_country,employee_city,employee_street,employee_street_number,emergency_contact,emergency_contact_number,relationship,title,monthly_salary,team,office,office_country,office_city,office_street,office_street_number
0,A2R5H9,Hunman,Jax,BE,Leuven,Grote Markt,9,Opie Hurst,+32-456-5556-84,Brother,CEO,$4500,Leadership,Leuven Office,BE,Leuven,Martelarenlaan,38.0
1,H8K0L6,Siff,Tara,GB,London,Baker Street,221,Wendy de Matteo,+44-020-5554-333,Sister,CFO,$4500,Leadership,WeWork Office,GB,London,Old Street,207.0
2,G4R7V0,Sagal,Gemma,US,New-York,Perry Street,66,John Newmark,+1-202-555-194,Husband,Business Developer,$3000,Sales,ESB Office,US,New York City,Fifth Avenue,350.0
3,M1Z7U9,Coates,Tig,FR,Paris,Rue de l'Université,7,Venus Noone,+1-202-555-0130,Wife,Office Manager,$2000,People Operations,NaN,NaN,NaN,NaN,NaN


In [117]:
# Reorder columns
new_column_order = ['employee_id', 'first_name', 'last_name', 'employee_country', 'employee_city', 'employee_street',
                    'employee_street_number', 'emergency_contact', 'emergency_contact_number', 'relationship',
                    'monthly_salary', 'team', 'title', 'office', 'office_country', 'office_city', 'office_street',
                    'office_street_number']

df_employees_renamed = df_employees_renamed[new_column_order]



# Set 'employee_id' as the index
df_employees_renamed.set_index('employee_id', inplace=True)

# Fill missing values in 'office' column with 'Remote'
df_employees_renamed['office'].fillna(value='Remote', inplace=True)


# Print the final DataFrame and export to CSV file
df_employees_renamed.head()
df_employees_renamed.to_csv('Exported_Employees.csv')

df_employees_renamed


,first_name,last_name,employee_country,employee_city,employee_street,employee_street_number,emergency_contact,emergency_contact_number,relationship,monthly_salary,team,title,office,office_country,office_city,office_street,office_street_number
employee_id,,,,,,,,,,,,,,,,,
A2R5H9,Jax,Hunman,BE,Leuven,Grote Markt,9,Opie Hurst,+32-456-5556-84,Brother,$4500,Leadership,CEO,Leuven Office,BE,Leuven,Martelarenlaan,38.0
H8K0L6,Tara,Siff,GB,London,Baker Street,221,Wendy de Matteo,+44-020-5554-333,Sister,$4500,Leadership,CFO,WeWork Office,GB,London,Old Street,207.0
G4R7V0,Gemma,Sagal,US,New-York,Perry Street,66,John Newmark,+1-202-555-194,Husband,$3000,Sales,Business Developer,ESB Office,US,New York City,Fifth Avenue,350.0
M1Z7U9,Tig,Coates,FR,Paris,Rue de l'Université,7,Venus Noone,+1-202-555-0130,Wife,$2000,People Operations,Office Manager,Remote,NaN,NaN,NaN,NaN
